## Modeling lobbying and stock trading jointly

The goal of this notebook is to propose a number of different models that jointly consider lobbying and stock trading. We will:
1. Fix a few baseline models, and see how they perform.
2. Choose a number of more complicated models, and explain the hyperparameter restrictions/sharing conditions we are imposing.
3. Model selection. In doing so, we will keep track of cross-validation error to try to minimize overfitting.
4. Identify our best choice, evaluate its performance, and try to improve it further.

First, let's import the basics here.

In [1]:
import numpy as np
import pandas as pd
import datetime
import ast
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()
from importlib import reload
import torch

# Silencing warnings 
import warnings
warnings.filterwarnings("ignore")

In [2]:
import scripts.data_extraction
reload(scripts.data_extraction)
from scripts.data_extraction import stock_and_lobbying_totals

In [3]:
cognates={'AGR' :  ['Agricultural Inputs', 'Farm & Heavy Construction Machinery', 'Farm Products'],
          'AUT' :  ['Auto & Truck Dealerships', 'Auto Manufacturers', 'Auto Parts', 'Recreational Vehicles'],
          'BEV' :  ['Beverages - Brewers', 'Beverages - Non-Alcoholic', 'Beverages - Wineries & Distilleries'], 
          'CHM' :  ['Chemicals', 'Specialty Chemicals'], 
          'DEF' :  ['Aerospace & Defense'], #Here I deviate from Paul's choice to include `AER` here as well
          'EDU' :  ['Education & Training Services'],
          'ENG' :  ['Solar'], #`ENG` code does not cover oil and gas, although many oil and gas companies lobby this code.
          'FOO' :  ['Confectioners', 'Food Distribution', 'Packaged Foods'],
          'FUE' :  ['Oil & Gas Drilling', 'Oil & Gas E&P','Oil & Gas Equipment & Services', 'Oil & Gas Integrated', 'Oil & Gas Midstream', 'Oil & Gas Refining & Marketing'],
          'INS' :  ['Insurance - Diversified', 'Insurance - Life', 'Insurance - Property & Casualty', 'Insurance - Reinsurance', 'Insurance - Specialty', 'Insurance Brokers'],
          'MAN' :  ['Metal Fabrication', 'Specialty Industrial Machinery', 'Steel', 'Textile Manufacturing'], #I added to this one to match choices Paul made
          'NAT' :  ['Aluminum', 'Copper', 'Lumber & Wood Production', 'Other Industrial Metals & Mining', 'Other Precious Metals & Mining', 'Silver'],
          'PHA' :  ['Drug Manufacturers - General', 'Drug Manufacturers - Specialty & Generic', 'Pharmaceutical Retailers'],
          'RES' :  ['REIT - Diversified', 'REIT - Industrial', 'REIT - Mortgage', 'REIT - Office', 'REIT - Specialty', 'Real Estate - Development', 'Real Estate - Diversified', 'Real Estate Services'],
          'RRR' :  ['Railroads'],
          'TEC' :  ['Telecom Services'],
          'TOB' :  ['Tobacco'],
          'UTI' :  ['Utilities - Diversified', 'Utilities - Independent Power Producers', 'Utilities - Regulated Water', 'Utilities - Renewable'] }

### (1) Baselines:

Here we want to work with multivariate baselines, as univariate baselines are already discussed in previous notebooks. In particular, extremely naive baseline models (e.g. naive mean or naive drift) are not relevant - they don't allow for even the most simple levels of interaction between the time series.

#### Naive multivariate linear model with white noise

Here's a baseline multivariate model that we would like to beat. 

### (2) Possible models, hyperparameter restrictions, and out of the box performance

Here I'll keep all the DARTS functionality and imports we need for our first pass at modeling.

In [4]:
from darts import TimeSeries, concatenate
from darts.utils.callbacks import TFMProgressBar
from darts.models import (ExponentialSmoothing, # our models
                          VARIMA,
                          XGBModel,
                          NBEATSModel,
                          TFTModel)
from darts.metrics import mape, smape, mae, msse
from darts.dataprocessing.transformers import Scaler #DARTS has a preprocessing function like sklearn's StandardScaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
import logging

logging.disable(logging.CRITICAL)

def generate_torch_kwargs():
    # run torch models on CPU, and disable progress bars for all model stages except training.
    return {
        "pl_trainer_kwargs": {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True)],
        }
    }

We'll make a train-validation split for this first pass with out of the box models, reserving the last 6 quarters for our validation set. Later, when we tune hyperparameters, I'll make choices using cross-validation and grid-search.

In [21]:
train_index =[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35]
val_index = [36, 37, 38, 39]

Note that our data (lobbying_total and stock_gross for various sectors) is:
- not seasonal;
- sometimes with visible trend;
- autoregressive (to a point).

Finally, here is a silly function we will call to prevent us from having to rewrite the same lines of code.

In [5]:
def fit_and_pred(model, training, validation):
    model.fit(training)
    forecast = model.predict(len(validation))
    return forecast

#### Exponential smoothing/Holt Winters

A double exponential smoothing model seems promising.

#### Vector autoregressive integrated moving average (VARIMA)

When using a VARIMA model, we are going to fix and share some hyperparameter choices with parameters across lobbying codes.

Because of this, we will assume that the $p$ and $d$ parameters of the $(p,d,q)$ (i.e. the autoregressiveness and how much differencing is done) will be shared over all sectors. We will restrict to $d\leq 1$ based on our EDA. We will allow $q$ to vary from sector to sector, and of course will fit the model to each sector (so that the coefficients/fit will vary sector-to-sector.)
In shorthand: we are using models of the form
$$ \mathrm{VARIMA}(p=const, d=const\leq 1, q) $$

The following is a function that runs a particular model for a given {issue code : sector} in `cognates`. It takes as input
- an issue code, `issue_code`,, in `cognates.keys()`;
- a model, `model` (eg. `model=VARIMA(p,d,q)`);
- a list or string of lobbying variables, `lobb_vars` (e.g. `lobby_vars=[lobbying_expenses, lobbying_income]`), and a list of stock variables, `stocks_vars`, which default to `lobb_vars=[lobbying_total]` and `stocks_vars=[stocks_gross]`;

and returns a Pandas dataframe of predictions for our model, trained over the training set, over the validation set. All data is scaled beforehand to improve model performance.

In [14]:
def predict(issue_code, model, lobb_vars=['lobbying_total'], stocks_vars=['stocks_gross']):
    if isinstance(lobb_vars, str):
        lobb_vars = [lobb_vars]
    if isinstance(stocks_vars, str):
        stocks_vars = [stocks_vars]
    df=stock_and_lobbying_totals(issue_code, cognates[issue_code])[lobb_vars+stocks_vars]
    totals=TimeSeries.from_dataframe(df)
    scaler=Scaler()
    totals_scaled =scaler.fit_transform(totals)
    train_scaled=totals_scaled[:-6]
    val_scaled=totals_scaled[-6:]
    scaled_pred = fit_and_pred(model, train_scaled, val_scaled)
    pred=scaler.inverse_transform(scaled_pred)
    code_preds=pd.DataFrame(pred.values())
    code_preds.index=df.index[-6:]
    code_preds.columns=[issue_code+"_"+item+"_pred" for item in lobb_vars+stocks_vars]
    metrics=[mape, smape, mae, msse]
    
    pred_error=pd.DataFrame()

    return code_preds

Here's a function that returns a dataframe of metrics measuring prediction error over the validation set. It takes as argument a dataframe of predictions, like those generated by the predict function above.

In [ ]:
def pred_error(df): 
    


In [15]:
trend=VARIMA(trend='t')
df=predict(issue_code='FOO', model=trend)

In [16]:
df

,FOO_lobbying_total_pred,FOO_stocks_gross_pred
time,,
2021-07-01,3.797978e+07,525520.230708
2021-10-01,3.866919e+07,563101.363988
2022-01-01,3.921832e+07,580570.680987
2022-04-01,3.973413e+07,596842.347350
2022-07-01,4.022613e+07,613003.816040
2022-10-01,4.069782e+07,629122.858055


In [34]:
lobb_var='lobbying_total'
stocks_var='stocks_gross'



VARIMA_preds=pd.DataFrame()
for issue_code in cognates.keys():
    df=stock_and_lobbying_totals(issue_code, cognates[issue_code])[[lobb_var, stocks_var]]
    totals=TimeSeries.from_dataframe(df)
    scaler=Scaler()
    totals_scaled=scaler.fit_transform(totals)
    train_scaled=totals[:-6]
    val_scaled=totals[-6:]
    model = VARIMA(trend='t')  #default setting for VARIMA are p=1, d=0, q=0. We try that here.
    scaled_pred = fit_and_pred(model, train_scaled, val_scaled)
    pred=scaler.inverse_transform(scaled_pred)
    VARIMA_preds[issue_code+'_lobb']=[l[0] for l in pred.values()]
    VARIMA_preds[issue_code+'_stocks']=[l[1] for l in pred.values()]
    VARIMA_preds.index=df.index[-6:]

In [35]:
VARIMA_preds

,AGR_lobb,AGR_stocks,AUT_lobb,AUT_stocks,BEV_lobb,BEV_stocks,CHM_lobb,CHM_stocks,DEF_lobb,DEF_stocks,...,RES_lobb,RES_stocks,RRR_lobb,RRR_stocks,TEC_lobb,TEC_stocks,TOB_lobb,TOB_stocks,UTI_lobb,UTI_stocks
time,,,,,,,,,,,,,,,,,,,,,
2021-07-01,3.979754e+15,4.745724e+11,1.702314e+15,2.806219e+12,1.772082e+13,5.111216e+11,2.163424e+15,7.957119e+11,1.592664e+16,7.372749e+11,...,6.354835e+14,4.500569e+11,3.270992e+14,8.325861e+10,1.457299e+16,4.278106e+12,2.524830e+14,1.776771e+11,4.914882e+14,2.971399e+10
2021-10-01,4.126089e+15,4.527775e+11,1.806860e+15,2.853924e+12,1.855594e+13,5.281356e+11,2.304990e+15,1.038839e+12,1.661295e+16,9.280322e+11,...,8.640361e+14,4.336577e+11,3.613993e+14,7.772951e+10,1.524917e+16,4.380986e+12,2.854341e+14,1.918980e+11,5.347352e+14,3.246989e+10
2022-01-01,4.275505e+15,4.723015e+11,1.907560e+15,2.919062e+12,1.940149e+13,5.443541e+11,2.408381e+15,1.172544e+12,1.730883e+16,9.248007e+11,...,1.039184e+15,4.599117e+11,4.018131e+14,7.892452e+10,1.593851e+16,4.504351e+12,3.144134e+14,2.033656e+11,5.779436e+14,3.342397e+10
2022-04-01,4.428370e+15,4.838822e+11,2.004158e+15,2.988459e+12,2.025785e+13,5.606196e+11,2.494383e+15,1.254348e+12,1.802054e+16,9.578099e+11,...,1.175359e+15,4.765706e+11,4.436146e+14,8.259262e+10,1.663917e+16,4.631029e+12,3.400364e+14,2.138294e+11,6.190230e+14,3.446631e+10
2022-07-01,4.584543e+15,4.970133e+11,2.097042e+15,3.059319e+12,2.112467e+13,5.769339e+11,2.572834e+15,1.311537e+12,1.874663e+16,9.842442e+11,...,1.283898e+15,4.931390e+11,4.850751e+14,8.711920e+10,1.735062e+16,4.758381e+12,3.629552e+14,2.235085e+11,6.583265e+14,3.549375e+10
2022-10-01,4.743983e+15,5.098649e+11,2.186671e+15,3.130990e+12,2.200157e+13,5.932953e+11,2.648383e+15,1.357054e+12,1.948710e+16,1.012063e+12,...,1.372707e+15,5.085037e+11,5.256032e+14,9.189495e+10,1.807255e+16,4.885998e+12,3.836976e+14,2.325565e+11,6.960461e+14,3.651310e+10


In [ ]:
fig, axs = plt.subplots(6,3, sharex=True)
fig.suptitle('VARIMA forecasts')
for i, issue_code in enumerate(cognates.keys()):
axs=axs.flatten()
axs[i]

#### XGBoost Model

#### Neural Basis Expansion Analysis Time Series Forecasting (N-BEATS)

#### Temporal fusion transformer (TFT)